In [1]:
import cv2
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [5]:
import os

base_dir = "datasettt"   # thay bằng đường dẫn folder dataset của bạn
rename_map = {
    "blocked": "0_blocked",
    "free_normal": "1_free_normal",
    "free_boost": "2_free_boost"
}

for old_name, new_name in rename_map.items():
    old_path = os.path.join(base_dir, old_name)
    new_path = os.path.join(base_dir, new_name)
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
        print(f"Đã đổi: {old_name} -> {new_name}")


Đã đổi: blocked -> 0_blocked
Đã đổi: free_normal -> 1_free_normal
Đã đổi: free_boost -> 2_free_boost


In [6]:
transform = transforms.Compose([
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('datasettt', transform=transform)

# Kiểm tra nhãn
print('Classes:', dataset.classes)


Classes: ['0_blocked', '1_free_normal', '2_free_boost']


In [7]:
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [len(dataset) - 50, 50]
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=True, num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False, num_workers=0
)


In [8]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)  # 3 lớp: blocked, normal, boost

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'model_resnet18_3class.pth'
best_accuracy = 0.0


In [10]:
for epoch in range(NUM_EPOCHS):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f'Epoch {epoch + 1}: Accuracy = {accuracy:.4f}')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), BEST_MODEL_PATH)


Epoch 1: Accuracy = 1.0000
Epoch 2: Accuracy = 1.0000
Epoch 3: Accuracy = 1.0000
Epoch 4: Accuracy = 1.0000
Epoch 5: Accuracy = 1.0000
Epoch 6: Accuracy = 0.9800
Epoch 7: Accuracy = 1.0000
Epoch 8: Accuracy = 1.0000
Epoch 9: Accuracy = 1.0000
Epoch 10: Accuracy = 1.0000
Epoch 11: Accuracy = 1.0000
Epoch 12: Accuracy = 1.0000
Epoch 13: Accuracy = 1.0000
Epoch 14: Accuracy = 1.0000
Epoch 15: Accuracy = 1.0000
Epoch 16: Accuracy = 1.0000
Epoch 17: Accuracy = 1.0000
Epoch 18: Accuracy = 1.0000
Epoch 19: Accuracy = 1.0000
Epoch 20: Accuracy = 1.0000
Epoch 21: Accuracy = 1.0000
Epoch 22: Accuracy = 1.0000
Epoch 23: Accuracy = 1.0000
Epoch 24: Accuracy = 1.0000
Epoch 25: Accuracy = 1.0000
Epoch 26: Accuracy = 1.0000
Epoch 27: Accuracy = 1.0000
Epoch 28: Accuracy = 1.0000
Epoch 29: Accuracy = 1.0000
Epoch 30: Accuracy = 1.0000
